In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
import cv2
import math
import numpy as np
from skimage import measure
from skimage.measure import regionprops
from skimage.morphology import opening, disk
from scipy.stats import mode
import matplotlib.pyplot as plt

In [0]:
PATIENT_ID = ['221','229','230','232','239','242','246','252','263']

In [4]:
ROOT_PATH = "/content/drive/My Drive/Breast Cancer Treatment/Numpy Arrays/Predicted Output/PR/Laddernet/13-05-20/"
NEGATIVE_ROOT_PATH = "/content/drive/My Drive/Breast Cancer Treatment/Numpy Arrays/Predicted Output/Negative/PR/"

FOUR_CLASS_PATH = []
NEGATIVE_PATH = []

for patient in (os.listdir(ROOT_PATH + 'Train/')):
  FOUR_CLASS_PATH.append(ROOT_PATH + "Train/" + patient)

for patient in (os.listdir(ROOT_PATH + 'Test/')):
  FOUR_CLASS_PATH.append(ROOT_PATH + "Test/" + patient)

for patient in (os.listdir(NEGATIVE_ROOT_PATH + 'Train/')):
  NEGATIVE_PATH.append(NEGATIVE_ROOT_PATH + "Train/" + patient)

for patient in (os.listdir(NEGATIVE_ROOT_PATH + 'Test/')):
  NEGATIVE_PATH.append(NEGATIVE_ROOT_PATH + "Test/" + patient)

print(len(FOUR_CLASS_PATH))
print(len(NEGATIVE_PATH))


9
9


In [0]:
def getCountUtil(image):

  # Opening operation
  selem = disk(15)
  opened = opening(image, selem)

  # Finding the regions/nuclei
  nuclei = measure.label(opened)
  props = regionprops(nuclei)

  # Removing small pixel patches
  if(len(props)==0):
    return 0
  new_props = list.copy(props)
  for i in range(len(props)):
    if props[i].area <= 400:
      new_props.pop(i)
  props = new_props

  # Caclulating average nuclei size
  if(len(props)==0):
    return 0
  avg_size = 0
  for i in range(len(props)):
    avg_size += props[i].area 
  avg_size /= len(props)

  # Splitting Overlapping Nuclei and getting count
  count = len(props)
  for i in range(len(props)):
    if(props[i].area > avg_size):
      count += math.floor(props[i].area/avg_size - 1)  
  
  return count

In [0]:
def getCount(image,intensity):
  new_image = np.copy(image)
  new_image[new_image!=intensity] = 0
  count = getCountUtil(new_image)
  return count

In [0]:
def getIntensityScore(strongCount,intermediateCount,weakCount,positiveCount,negativeCount):
  IS = 0
  weak_percent = weakCount/(positiveCount + negativeCount)
  intermediate_percent = intermediateCount/(positiveCount + negativeCount)
  strong_percent = strongCount/(positiveCount + negativeCount)

  IS = weak_percent + intermediate_percent * 2 + strong_percent * 3
  # maxCount = max(strongCount,1.5 * intermediateCount,2 * weakCount)
  # if(maxCount == 0):
  #   IS = 0              # No Staining
  # elif(maxCount == 2* weakCount):
  #   IS = 1              # Weak Positive Staining
  # elif(maxCount == 1.5 *intermediateCount):
  #   IS = 2              # Moderate Positive Staining
  # else:
  #   IS = 3              # Strong Positive Staining
  return math.floor(IS)

In [0]:
def getProliferationScore(positiveCount,negativeCount):
  PS = 0
  stainedCellPercentage = positiveCount / (positiveCount + negativeCount) * 100.0
  if(stainedCellPercentage == 0):
    PS = 0            # No cells are ER positive
  elif(stainedCellPercentage < 1):
    PS = 1            # <1% cells are ER positive
  elif(stainedCellPercentage <= 10):
    PS = 2            # 1-10% cells are ER positive
  elif(stainedCellPercentage <= 33):
    PS = 3            # 11-33% cells are ER positive
  elif(stainedCellPercentage <= 66):
    PS = 4            # 34-66% cells are ER positive
  else:
    PS = 5            # 67-100% cells are ER positive
  return PS

In [0]:
def getAllredScore(IS,PS):
  return IS + PS      # Allred's score = Intensity Score + Proliferation Score

In [0]:
def getResult(score):
  result = 0
  if(score<=2):
    result = 0      # 0-2: ER negative
  else:
    result = 1      # 3-8: ER positive
  return result

In [0]:
def ScoreCalculation(mask,negativeMask):
  negativeCount = getCount(negativeMask,255)
  strongCount = getCount(mask,255)
  intermediateCount = getCount(mask,170)
  weakCount = getCount(mask,85)

  print("Negative : " , negativeCount)
  print("Strong : " , strongCount)
  print("Intermediate : " , intermediateCount)
  print("Weak : " , weakCount)

  positiveCount = strongCount + intermediateCount + weakCount

  IS = getIntensityScore(strongCount,intermediateCount,weakCount,positiveCount,negativeCount)
  PS = getProliferationScore(positiveCount,negativeCount)

  AS = getAllredScore(IS,PS)
  print("Score : " , AS , "\n")

  result = getResult(AS)
  return result

In [0]:
# Patient-wise result
def patientWiseResultUtil(fourClassNumpyPath,negativeNumpyPath):
  mask = np.load(fourClassNumpyPath)
  negativeMask = np.load(negativeNumpyPath)
  result = np.zeros(len(mask), dtype=int)
  
  for i in range(len(mask)):
    result[i] = ScoreCalculation(mask[i],negativeMask[i])
  return (mode(result).mode)[0]

In [0]:
def patientWiseResult(PatientID):

  path4Class = ""
  pathNegative = ""

  for p in FOUR_CLASS_PATH:
     num = (p.split('/')[-1]).split('.')[0]
     if(num==PatientID):
       path4Class = p
       break

  for p in NEGATIVE_PATH:
     num = (p.split('/')[-1]).split('.')[0]
     if(num==PatientID):
       pathNegative = p
       break

  print("PATIENT ", PatientID,"\n")
  val = patientWiseResultUtil(path4Class, pathNegative)
  if(val):
    print("IHC ", PatientID, " Prognosis : PR+")
  else:
    print("IHC ", PatientID, " Prognosis : PR-")

In [15]:
for id in PATIENT_ID:
  patientWiseResult(id)

PATIENT  221 

Negative :  170
Strong :  0
Intermediate :  0
Weak :  6
Score :  2 

Negative :  176
Strong :  0
Intermediate :  0
Weak :  0
Score :  0 

Negative :  218
Strong :  0
Intermediate :  0
Weak :  1
Score :  1 

Negative :  150
Strong :  0
Intermediate :  0
Weak :  6
Score :  2 

Negative :  177
Strong :  0
Intermediate :  0
Weak :  2
Score :  2 

Negative :  175
Strong :  0
Intermediate :  0
Weak :  1
Score :  1 

Negative :  133
Strong :  0
Intermediate :  0
Weak :  1
Score :  1 

Negative :  130
Strong :  0
Intermediate :  0
Weak :  5
Score :  2 

Negative :  163
Strong :  0
Intermediate :  0
Weak :  1
Score :  1 

Negative :  172
Strong :  0
Intermediate :  0
Weak :  0
Score :  0 

IHC  221  Prognosis : PR-
PATIENT  229 

Negative :  79
Strong :  0
Intermediate :  0
Weak :  5
Score :  2 

Negative :  65
Strong :  0
Intermediate :  0
Weak :  0
Score :  0 

Negative :  92
Strong :  0
Intermediate :  0
Weak :  3
Score :  2 

Negative :  69
Strong :  0
Intermediate :  0
Weak 